In [35]:
import tensorflow as tf
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input,Dense, Subtract, Add, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from keras.constraints import maxnorm
from sklearn.linear_model import Lasso
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from math import sqrt


data = pd.read_excel("C:/Users/ERIC/Desktop/PML-06MTP-115/06MTP-115.xlsm", 1)

# Training dataset
x = data.iloc[:, [1,2,26,50]].values
y = data.iloc[:, 74].values

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state = 42) #try random_state = 42

#Standarization of the data
sc = StandardScaler()
std_x_train = sc.fit_transform(x_train)
std_x_test = sc.fit_transform(x_test)

input_tensor = Input(shape=(x.shape[1],))
layer_drop = Dropout(0.3)

hidden_layer = Dense(1000, activation = 'relu', kernel_constraint=maxnorm(3),#Make better results with Dropout
                     kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),#To avoid Over-fitting
                     activity_regularizer=regularizers.l1(0.0001))(input_tensor) #Regularize the wieghts to have better accuracy
layer_drop2 = Dropout(0.3)

hidden_layer1 = Dense(500, activation = 'relu',kernel_constraint=maxnorm(3),#Make better results with Dropout
                     kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),#To avoid Over-fitting
                     activity_regularizer=regularizers.l1(0.0001))(hidden_layer) #Regularize the wieghts to have better accuracy
layer_drop3 = Dropout(0.3)

hidden_layer2 = Dense(50, activation = 'relu', kernel_constraint=maxnorm(3),#Make better results with Dropout
                     kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),#To avoid Over-fitting
                     activity_regularizer=regularizers.l1(0.0001))(hidden_layer1) #Regularize the wieghts to have better accuracy
layer_drop4 = Dropout(0.3)

output_tensor = Dense(1)(hidden_layer2)

model = Model(input_tensor, output_tensor)

model.compile(optimizer= 'adam',  loss='mean_absolute_error', metrics = ['mean_absolute_error'])

#monitor = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 5, verbose = 1, mode = 'auto', 
                        #restore_best_weights = True)

model.summary()

model.fit(x_train,
          y_train,
          #validation_data = (x_test, y_test),
          #callbacks = [monitor],
          epochs=100,
          batch_size = 3,
          validation_split = 0.1,
          verbose=True)

pred = model.predict(x_test)
score = np.sqrt(mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_44 (Dense)             (None, 1000)              5000      
_________________________________________________________________
dense_45 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_46 (Dense)             (None, 50)                25050     
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 51        
Total params: 530,601
Trainable params: 530,601
Non-trainable params: 0
_________________________________________________________________
Train on 12 samples, validate on 2 samples
Epoch 1/100
12/12 [==============================] - 3s 257ms/sample - loss

12/12 [==============================] - 0s 7ms/sample - loss: 168.5705 - mean_absolute_error: 146.9997 - val_loss: 162.6003 - val_mean_absolute_error: 143.0743
Epoch 43/100
12/12 [==============================] - 0s 7ms/sample - loss: 106.9059 - mean_absolute_error: 85.9311 - val_loss: 54.1045 - val_mean_absolute_error: 35.2435
Epoch 44/100
12/12 [==============================] - 0s 6ms/sample - loss: 148.9962 - mean_absolute_error: 128.6080 - val_loss: 48.5726 - val_mean_absolute_error: 29.9482
Epoch 45/100
12/12 [==============================] - 0s 6ms/sample - loss: 116.4761 - mean_absolute_error: 96.1849 - val_loss: 142.8291 - val_mean_absolute_error: 124.0887
Epoch 46/100
12/12 [==============================] - 0s 6ms/sample - loss: 122.0737 - mean_absolute_error: 101.7365 - val_loss: 123.6695 - val_mean_absolute_error: 105.1864
Epoch 47/100
12/12 [==============================] - 0s 6ms/sample - loss: 117.2315 - mean_absolute_error: 97.3946 - val_loss: 44.8551 - val_mean_ab

Epoch 90/100
12/12 [==============================] - 0s 6ms/sample - loss: 93.0876 - mean_absolute_error: 77.2563 - val_loss: 48.9139 - val_mean_absolute_error: 34.3237
Epoch 91/100
12/12 [==============================] - 0s 6ms/sample - loss: 108.4424 - mean_absolute_error: 92.5351 - val_loss: 122.1423 - val_mean_absolute_error: 107.3558
Epoch 92/100
12/12 [==============================] - 0s 6ms/sample - loss: 110.4532 - mean_absolute_error: 94.6240 - val_loss: 27.7961 - val_mean_absolute_error: 13.4060
Epoch 93/100
12/12 [==============================] - 0s 6ms/sample - loss: 102.2944 - mean_absolute_error: 86.7172 - val_loss: 50.3567 - val_mean_absolute_error: 35.8926
Epoch 94/100
12/12 [==============================] - 0s 7ms/sample - loss: 82.5898 - mean_absolute_error: 66.9300 - val_loss: 111.2634 - val_mean_absolute_error: 96.6837
Epoch 95/100
12/12 [==============================] - 0s 7ms/sample - loss: 98.4918 - mean_absolute_error: 82.7182 - val_loss: 104.3180 - val_me